In [121]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy

In [139]:
# constants
n_nodes = 10
n_edges = 2
seed = 1234
N_SIM = 100c

MEAN_RANGE = (-10, 10)
STD_RANGE = (0.1, 5.0)
MEAN_ERROR_RANGE = (0.02, 0.3)
STD_ERROR_RANGE = (0.001, 0.1)

This notebook demonstrates the first attempt in the Data Generation Process. 

To generate a network I used Barabasi-Albert model that is commonly used for modeling real-world systems, for instance friendhip groups or high school networks. It generates networks with a power-law degree distribution, resulting in some nodes have a much larger number of connections than others.

To start with, we need to assume the type of network effect. Here I assume relational network effect, meaning that the outcome of te unit depends on its covariates, as well as their neighbours' outcome and covariates. There also could be a positional network effect, which depicts an effect on a network level, meaning that unit's position in network affects outcome and structural effect that depicts effect of the network as a whole. For simplicity I don't assume this. 

Additionally, we can simulate whether the influence of every node is the same by normalizing the adjacency matrix. Hovewer, in real-world scenarios I find this assumption too simplistic. 

To simulate these scenarios we can apply spatial lag model approaches. The value of the outcome of one unit depends on the value of its neighbours, leading to dependencies between neighbours. This means that a system of equations has to be solved simultaneously: $$ y = X\beta + \rho Wy + \epsilon $$

Solving for y:

$$y = (I - \rho W)^{-1}X\beta + (I - \rho W)^{-1}\epsilon$$

Then, we can simulate treatment $Z$, which results in the following model: $$y = (I - \rho W)^{-1}X\beta_{1} + (I - \rho W)^{-1} (\beta_{2}*Z) + (I - \rho W)^{-1}\epsilon$$

In [151]:
def generate_random_mean_std(n_features, mean_range=MEAN_RANGE, std_range=STD_RANGE):
    means = np.random.uniform(mean_range[0], mean_range[1], n_features)
    stds = np.random.uniform(std_range[0], std_range[1], n_features)
    return means, stds

def generate_random_beta_matrix(n_features):
    means, stds = generate_random_mean_std(n_features)
    beta = np.random.normal(means, stds, n_features)
    return beta

def generate_random_data(n_nodes, n_features):
    means, stds = generate_random_mean_std(n_features)
    data = np.zeros((n_nodes, n_features))
    
    for i in range(n_features):
        data[:, i] = np.random.normal(means[i], stds[i], n_nodes)
        
    return data

def generate_random_error(n_nodes, mean_range=MEAN_ERROR_RANGE, std_range=STD_ERROR_RANGE):
    mean, std = generate_random_mean_std(n_features=1, mean_range=mean_range, std_range=std_range)
    error = np.random.normal(mean, std, n_nodes)

    return error

def get_group_data(n_nodes, share_treatment=0.5, random=True, covariates=0):
    if random:
        pass
    else:
        n_treatment = int(n_nodes * share_treatment)
        n_control = n_nodes - n_treatment 
        group = np.array([1] * n_treatment + [0] * n_control)
    return group

In [168]:
def define_outcome_SAR(
    G,
    n_nodes,
    covariates,
    betas,
    epsilon,
    neighbour_influence,
    row_normalized=False,
    group_data=0):
    adj_matrix = nx.to_numpy_array(G)
    if row_normalized:
        adj_matrix = adj_matrix / adj_matrix.sum(axis=1, keepdims=True)
    I = np.eye(n_nodes)
    weight = scipy.linalg.inv(I - neighbour_influence*adj_matrix)
    if group_data:
        y = weight@covariates@betas + weight@epsilon + weight@group_data*treatment_effect
    else:
        y = weight@covariates@betas + weight@epsilon

    return y

In [169]:
def data_simulation(
    n_sim,
    n_nodes,
    n_edges,
    n_features,
    mean_range_cov,
    std_range_cov,
    mean_range_error,
    std_range_error,
    share_treatment=0.5,
    random=True,
    seed=seed
):
    outcome_dict = {i: [] for i in n_sim}
    for i in n_sim:
        G = nx.barabasi_albert_graph(n=n_nodes, m=n_edges, seed=seed)
        means, stds = generate_random_mean_std(n_features, mean_range=mean_range_cov, std_range=std_range_cov)
        betas = generate_random_beta_matrix(n_features)
        data = generate_random_data(n_nodes, n_features)
        error = generate_random_error(n_nodes, mean_range=mean_range_error, std_range=std_range_error)
        group_assignment = get_group_data(n_nodes, share_treatment=0.5, random=True, covariates=0)
        outcome = define_outcome_SAR(
            G,
            n_nodes,
            covariates,
            betas,
            epsilon,
            neighbour_influence,
            group_assignment)
        outcome_dict[i] = outcome
    
    return outcome

The treatment assignment also can differs. For example, it can be random but as we are modelling observational study, this assumption don't apply. I assume that the treatment of the unit depends on its covariates. There are several ways to model this, using logistic regression model or some linear threshold. 

TODO:
1. Simulate different treatment assignments
2. Simulate effect estimation: naive, matching
3. Add time 